# 1. Подготовка данных

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import xgboost as xgb
import numpy as np
import optuna
import mlflow
import joblib

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, f1_score, balanced_accuracy_score
from sklearn.svm import SVC
from imblearn.pipeline import Pipeline

mlflow.set_tracking_uri("http://127.0.0.1:8000")
client=mlflow.MlflowClient(tracking_uri='http://127.0.0.1:8000')

In [2]:
df = pd.read_csv('Churn.csv')

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
# проверим наличие явных дубликатов:
df.duplicated().sum()

0

In [6]:
# посмотрим уникальные значения в столбце Geography:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
# посмотрим уникальные значения в столбце Tenure:
df['Tenure'].value_counts().sort_values(ascending=False)

Tenure
1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: count, dtype: int64

In [8]:
# посмотрим уникальные значения в столбце NumOfProducts:
df['NumOfProducts'].value_counts()

NumOfProducts
1    5084
2    4590
3     266
4      60
Name: count, dtype: int64

In [9]:
# посмотрим уникальные значения в столбце HasCrCard:
df['HasCrCard'].value_counts()

HasCrCard
1    7055
0    2945
Name: count, dtype: int64

In [10]:
# посмотрим уникальные значения в столбце IsActiveMember:
df['IsActiveMember'].value_counts()

IsActiveMember
1    5151
0    4849
Name: count, dtype: int64

In [11]:
# посчитаем классы в целевом признаке:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

Можно отметить, что дубликаты в датасете отсутствуют, в столбце Tenure есть около тысячи пропусков, а также несколько столбцов лучше вообще удалить, из них:

- RowNumber - это индекс строки в данных, важной информации не содержит;
- CustomerId - уникальный идентификатор клиента, по нему мы проверили дубликаты и теперь его можно удалить;
- Surname - фамилия, которая тоже не имеет связи с оттоком клиентов.

После проверки датасета на сбалансированность классов в таргете мы видим, что объектов отрицательного класса намного больше (почти в четыре раза), но все равно уходит примерно 20% клиентов. В столбце Geography находятся всего 3 уникальных значения: France, Germany и Spain. В столбце Tenure (сколько лет человек является клиентом банка) есть 909 пропусков. Заполним их значением -1, добавив логический параметр add_indicator=True. Есть несколько бинарных признаков (HasCrCard, IsActiveMember), которые затем выделим в отдельную категорию признаков.

In [12]:
# удалим ненужные столбцы:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [13]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [15]:
# сохраним признаки и целевой признак в отдельных переменных:
y = df['Exited']
X = df.drop('Exited', axis=1)

In [16]:
# разобьем исходные данные на обучающую и тестовую выборки в соотношении 4:1:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345, stratify=y)

In [17]:
# посмотрим на размеры выборок:
print('Размер обучающей выборки:', X_train.shape)
print('Размер тестовой выборки:', X_test.shape)

Размер обучающей выборки: (8000, 10)
Размер тестовой выборки: (2000, 10)


In [18]:
# внесем в переменные списки все численные и категориальные признаки:
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_train.select_dtypes(exclude=['object']).columns.tolist()
binary_features = ['HasCrCard', 'IsActiveMember']
numeric_features = list(set(numeric_features) - set(binary_features))

In [19]:
numeric_features

['Balance', 'NumOfProducts', 'Age', 'CreditScore', 'Tenure', 'EstimatedSalary']

In [20]:
categorical_features

['Geography', 'Gender']

In [21]:
binary_features

['HasCrCard', 'IsActiveMember']

# 2. Обучение модели без учёта дисбаланса классов (XGBoost)

Для автоматизации процесса подготовим численный и категориальный пайплайны.
- Для категориальных признаков (Geography, Gender) используем One Hot Encoding.
- Для заполнения пропусков используем SimpleImputer.
- Выполним трансформацию Йео-Джонсона.
- 
- Объединим пайплайны классом ColumnTransformer.

In [22]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()) 
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)

], remainder='passthrough')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBClassifier(random_state=12345))
])

In [23]:
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('enc',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Geography', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value=-1)),
                                                                  ('pt',
                                                                   PowerTransformer())]),
                                                  ['Balance', 'NumOfProducts',
                                                   'Age', 'CreditScore',
                                                   'Tenure',
                                                   'EstimatedSala...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=12345, ...))])

In [24]:
def optuna_optimize(trial):
    param_grid = {
        'model__n_estimators': trial.suggest_int('model__n_estimators', 50, 250),
        'model__max_depth': trial.suggest_int('model__max_depth', 2, 10),
        'model__eta': trial.suggest_float('model__eta', 0.01, 1.5)
    }

    pipe.set_params(**param_grid)
    score = cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [25]:
study_xgb = optuna.create_study(study_name="XGBoost Optuna Optimization Imbalanced",
                                direction='maximize')

[I 2023-11-27 17:56:48,892] A new study created in memory with name: XGBoost Optuna Optimization Imbalanced


In [26]:
study_xgb.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 17:56:56,155] Trial 0 finished with value: 0.5787073296259104 and parameters: {'model__n_estimators': 226, 'model__max_depth': 2, 'model__eta': 0.9035716221661731}. Best is trial 0 with value: 0.5787073296259104.
[I 2023-11-27 17:57:09,862] Trial 1 finished with value: 0.5627959974249825 and parameters: {'model__n_estimators': 190, 'model__max_depth': 7, 'model__eta': 0.16576191371248003}. Best is trial 0 with value: 0.5787073296259104.
[I 2023-11-27 17:57:21,451] Trial 2 finished with value: 0.566714580678235 and parameters: {'model__n_estimators': 100, 'model__max_depth': 9, 'model__eta': 0.13037834470286025}. Best is trial 0 with value: 0.5787073296259104.
[I 2023-11-27 17:57:31,115] Trial 3 finished with value: 0.5621566544335165 and parameters: {'model__n_estimators': 71, 'model__max_depth': 9, 'model__eta': 0.16359750708798868}. Best is trial 0 with value: 0.5787073296259104.
[I 2023-11-27 17:57:36,714] Trial 4 finished with value: 0.5220454592296531 and parameters:

In [27]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb.best_value))

Наилучшие значения гиперпараметров XGBoost: {'model__n_estimators': 91, 'model__max_depth': 2, 'model__eta': 0.6813624023889251}
f1_score на обучающем наборе: 0.592


In [28]:
with mlflow.start_run(nested=True):

    best_params = study_xgb.best_params
    best_score = study_xgb.best_value
    mlflow.log_params(best_params)

    pipe.set_params(**best_params)
    pipe.fit(X_train, y_train)
    pred_proba = pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba)),
        'f1': f1_score(y_test, np.round(pred_proba))
    })

    mlflow.sklearn.log_model(pipe, 'xgb_pipeline_imbalanced')

# 3. Обучение моделей с учётом дисбаланса классов
## 3.1 Назначение разных весов классам (cost-sensitive learning)
### 3.1.1 Модель XGBoost

In [29]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)

], remainder='passthrough')

In [30]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'eta': trial.suggest_float('eta', 0.01, 1.5)
    }

    estimator = xgb.XGBClassifier(**param, 
                                  scale_pos_weight=3.91,
                                  random_state=12345)
    
    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])

    score = cross_val_score(xgb_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [31]:
study_xgb_cw = optuna.create_study(study_name="XGBoost Optuna Optimization Class Weight",
                                direction='maximize')

[I 2023-11-27 18:00:28,673] A new study created in memory with name: XGBoost Optuna Optimization Class Weight


In [32]:
study_xgb_cw.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 18:00:31,301] Trial 0 finished with value: 0.5549611632997264 and parameters: {'n_estimators': 118, 'max_depth': 7, 'eta': 1.1071110105706021}. Best is trial 0 with value: 0.5549611632997264.
[I 2023-11-27 18:00:33,545] Trial 1 finished with value: 0.5372267842712488 and parameters: {'n_estimators': 242, 'max_depth': 3, 'eta': 1.328303598241657}. Best is trial 0 with value: 0.5549611632997264.
[I 2023-11-27 18:00:37,207] Trial 2 finished with value: 0.5521646017939198 and parameters: {'n_estimators': 217, 'max_depth': 9, 'eta': 1.1564266612634584}. Best is trial 0 with value: 0.5549611632997264.
[I 2023-11-27 18:00:39,355] Trial 3 finished with value: 0.5829922934952096 and parameters: {'n_estimators': 81, 'max_depth': 7, 'eta': 0.2917528977750594}. Best is trial 3 with value: 0.5829922934952096.
[I 2023-11-27 18:00:42,810] Trial 4 finished with value: 0.5607949099927797 and parameters: {'n_estimators': 167, 'max_depth': 9, 'eta': 0.6797800387123878}. Best is trial 3 with

In [33]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb_cw.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb_cw.best_value))

Наилучшие значения гиперпараметров XGBoost: {'n_estimators': 104, 'max_depth': 2, 'eta': 0.2568437923372118}
f1_score на обучающем наборе: 0.611


In [34]:
with mlflow.start_run(nested=True):

    best_params = study_xgb_cw.best_params
    best_score = study_xgb_cw.best_value
    mlflow.log_params(best_params)

    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', xgb.XGBClassifier(**best_params,
                                    scale_pos_weight=3.91,
                                    random_state=12345))
    ])

    xgb_pipe.fit(X_train, y_train) 

    pred_proba = xgb_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(xgb_pipe, 'xgb_pipeline_class_weight')

### 3.1.2 Случайный лес

In [35]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8)
    }

    estimator = RandomForestClassifier(**param, 
                                       class_weight='balanced',
                                       random_state=12345)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])

    score = cross_val_score(rf_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [36]:
study_rf = optuna.create_study(study_name="RandomForest Optuna Optimization Class Weight",
                                direction='maximize')

[I 2023-11-27 18:03:48,575] A new study created in memory with name: RandomForest Optuna Optimization Class Weight


In [37]:
study_rf.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 18:04:01,650] Trial 0 finished with value: 0.613835794662274 and parameters: {'n_estimators': 132, 'max_depth': 10, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.613835794662274.
[I 2023-11-27 18:04:16,017] Trial 1 finished with value: 0.5724933917477408 and parameters: {'n_estimators': 294, 'max_depth': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.613835794662274.
[I 2023-11-27 18:04:33,379] Trial 2 finished with value: 0.6170594099284914 and parameters: {'n_estimators': 218, 'max_depth': 8, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.6170594099284914.
[I 2023-11-27 18:04:43,168] Trial 3 finished with value: 0.574600214987844 and parameters: {'n_estimators': 196, 'max_depth': 4, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.6170594099284914.
[I 2023-11-27 18:04:57,717] Trial 4 finished with value: 0.5896815069217184 and parameters: {'n_estimators': 263, 'max_depth': 5, 'min_samples_leaf': 6}. Best is trial 2 with value: 0.61705940992

In [38]:
print("Наилучшие значения гиперпараметров Random Forest: {}".format(study_rf.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_rf.best_value))

Наилучшие значения гиперпараметров Random Forest: {'n_estimators': 274, 'max_depth': 8, 'min_samples_leaf': 3}
f1_score на обучающем наборе: 0.618


In [39]:
with mlflow.start_run(nested=True):

    best_params = study_rf.best_params
    best_score = study_rf.best_value
    mlflow.log_params(best_params)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestClassifier(**best_params,
                                         class_weight='balanced',
                                         random_state=12345))
    ])

    rf_pipe.fit(X_train, y_train)

    pred_proba = rf_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(rf_pipe, 'rf_pipeline_class_weight')

### 3.1.3 Логистическая регрессия

In [40]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()),
    ('scal', StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [41]:
def optuna_optimize(trial):
    param = {
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga'])
    }
    
    if param['solver'] == 'lbfgs':
        param['penalty'] = 'l2' 

    if param['solver'] == 'saga':
        param['penalty'] = 'elasticnet' 
        param['C'] = trial.suggest_float('C', 0.01, 100) 
        param['l1_ratio'] = trial.suggest_float('l1_ratio', 0, 1)
        param['max_iter'] = 1000 
        
    estimator = LogisticRegression(**param,  
                                   class_weight='balanced',
                                   random_state=12345)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])
    
    score = cross_val_score(log_reg_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [42]:
# optuna.logging.set_verbosity(optuna.logging.WARNING) # i do not want to see trail information
study_log_reg = optuna.create_study(study_name="LogisticRegression Optuna Optimization Class Weight",
                                direction='maximize')

[I 2023-11-27 18:31:25,437] A new study created in memory with name: LogisticRegression Optuna Optimization Class Weight


In [43]:
study_log_reg.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 18:31:26,837] Trial 0 finished with value: 0.4910843408008952 and parameters: {'solver': 'saga', 'C': 48.64114241482636, 'l1_ratio': 0.8072033231067617}. Best is trial 0 with value: 0.4910843408008952.
[I 2023-11-27 18:31:28,062] Trial 1 finished with value: 0.4910843408008952 and parameters: {'solver': 'saga', 'C': 62.01297164567831, 'l1_ratio': 0.5183249821180206}. Best is trial 0 with value: 0.4910843408008952.
[I 2023-11-27 18:31:29,274] Trial 2 finished with value: 0.4910843408008952 and parameters: {'solver': 'saga', 'C': 44.72418813282303, 'l1_ratio': 0.028488104730567065}. Best is trial 0 with value: 0.4910843408008952.
[I 2023-11-27 18:31:30,478] Trial 3 finished with value: 0.4910843408008952 and parameters: {'solver': 'saga', 'C': 45.99247173154098, 'l1_ratio': 0.16027747283468285}. Best is trial 0 with value: 0.4910843408008952.
[I 2023-11-27 18:31:31,688] Trial 4 finished with value: 0.4910843408008952 and parameters: {'solver': 'saga', 'C': 89.95473463365998

In [44]:
print("Наилучшие значения гиперпараметров Logistic Regression: {}".format(study_log_reg.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_log_reg.best_value))

Наилучшие значения гиперпараметров Logistic Regression: {'solver': 'saga', 'C': 48.64114241482636, 'l1_ratio': 0.8072033231067617}
f1_score на обучающем наборе: 0.491


In [45]:
with mlflow.start_run(nested=True):

    best_params = study_log_reg.best_params
    best_score = study_log_reg.best_value
    mlflow.log_params(best_params)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(**best_params,
                                     class_weight='balanced',
                                     random_state=12345))
    ])

    log_reg_pipe.fit(X_train, y_train)

    pred_proba = log_reg_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(log_reg_pipe, 'log_reg_pipeline_class_weight')

### 3.1.4 Метод опорных векторов

In [46]:
def optuna_optimize(trial):

    param = {
        'C': trial.suggest_float('C', 0.001, 200)  # какие значения может принимать С 
    }
 
    estimator = SVC(**param, 
                    class_weight='balanced',
                    random_state=12345,
                    probability=True)
    
    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])
    
    score = cross_val_score(svc_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [47]:
study_svc = optuna.create_study(study_name="SVC Optuna Optimization Class Weight",
                                    direction='maximize')

[I 2023-11-27 18:33:35,137] A new study created in memory with name: SVC Optuna Optimization Class Weight


In [48]:
study_svc.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 18:35:26,996] Trial 0 finished with value: 0.5428718808424785 and parameters: {'C': 43.18733270095023}. Best is trial 0 with value: 0.5428718808424785.
[I 2023-11-27 18:37:57,194] Trial 1 finished with value: 0.5222251165833668 and parameters: {'C': 92.59268626953713}. Best is trial 0 with value: 0.5428718808424785.
[I 2023-11-27 18:41:17,510] Trial 2 finished with value: 0.5122732258725616 and parameters: {'C': 173.54116214747552}. Best is trial 0 with value: 0.5428718808424785.
[I 2023-11-27 18:42:52,742] Trial 3 finished with value: 0.5545596719380078 and parameters: {'C': 24.04447129961635}. Best is trial 3 with value: 0.5545596719380078.
[I 2023-11-27 18:45:59,034] Trial 4 finished with value: 0.5114930159050126 and parameters: {'C': 149.75692923081712}. Best is trial 3 with value: 0.5545596719380078.
[I 2023-11-27 18:48:13,095] Trial 5 finished with value: 0.5336372873845594 and parameters: {'C': 71.1741081338911}. Best is trial 3 with value: 0.5545596719380078.
[I 

In [49]:
print("Наилучшие значения гиперпараметров SVC: {}".format(study_svc.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_svc.best_value))

Наилучшие значения гиперпараметров SVC: {'C': 2.393187279929207}
f1_score на обучающем наборе: 0.579


In [51]:
with mlflow.start_run(nested=True):

    best_params = study_svc.best_params
    best_score = study_svc.best_value
    mlflow.log_params(best_params)

    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', SVC(**best_params,
                      class_weight='balanced',
                      random_state=12345,
                      probability=True))
    ])

    svc_pipe.fit(X_train, y_train)

    pred_proba = svc_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(svc_pipe, 'svc_pipeline_class_weight')

## 3.2 Генерация синтетических данных (SMOTE)
### 3.2.1 Модель XGBoost

In [52]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [53]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'eta': trial.suggest_float('eta', 0.01, 1.5)
    }

    estimator = xgb.XGBClassifier(**param,
                                  random_state=12345)

    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(xgb_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [54]:
study_xgb_smote = optuna.create_study(study_name="XGBoost Optuna Optimization SMOTE",
                                direction='maximize')

[I 2023-11-27 21:23:49,524] A new study created in memory with name: XGBoost Optuna Optimization SMOTE


In [55]:
study_xgb_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 21:23:59,829] Trial 0 finished with value: 0.5898452232180633 and parameters: {'n_estimators': 134, 'max_depth': 6, 'eta': 0.18264227122182547}. Best is trial 0 with value: 0.5898452232180633.
[I 2023-11-27 21:24:03,300] Trial 1 finished with value: 0.603950534712141 and parameters: {'n_estimators': 190, 'max_depth': 3, 'eta': 0.287909568360685}. Best is trial 1 with value: 0.603950534712141.
[I 2023-11-27 21:24:20,790] Trial 2 finished with value: 0.5591783265483217 and parameters: {'n_estimators': 116, 'max_depth': 10, 'eta': 0.7906725167664105}. Best is trial 1 with value: 0.603950534712141.
[I 2023-11-27 21:24:42,979] Trial 3 finished with value: 0.5760625055682145 and parameters: {'n_estimators': 189, 'max_depth': 7, 'eta': 0.16877814376297545}. Best is trial 1 with value: 0.603950534712141.
[I 2023-11-27 21:24:54,417] Trial 4 finished with value: 0.5503809869865706 and parameters: {'n_estimators': 206, 'max_depth': 4, 'eta': 0.8619150526613939}. Best is trial 1 with

In [56]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb_smote.best_value))

Наилучшие значения гиперпараметров XGBoost: {'n_estimators': 171, 'max_depth': 3, 'eta': 0.11686014730847895}
f1_score на обучающем наборе: 0.626


In [57]:
with mlflow.start_run(nested=True):

    best_params = study_xgb_smote.best_params
    best_score = study_xgb_smote.best_value
    mlflow.log_params(best_params)

    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', xgb.XGBClassifier(**best_params,
                                    random_state=12345))
    ])

    xgb_pipe.fit(X_train, y_train)

    pred_proba = xgb_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(xgb_pipe, 'xgb_pipeline_smote')

### 3.2.2 Случайный лес

In [58]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8)
    }

    estimator = RandomForestClassifier(**param,
                                       random_state=12345)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(rf_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [59]:
study_rf_smote = optuna.create_study(study_name="RandomForest Optuna Optimization SMOTE",
                               direction='maximize')

[I 2023-11-27 21:29:53,137] A new study created in memory with name: RandomForest Optuna Optimization SMOTE


In [60]:
study_rf_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 21:30:29,643] Trial 0 finished with value: 0.6088191058920795 and parameters: {'n_estimators': 207, 'max_depth': 10, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.6088191058920795.
[I 2023-11-27 21:30:34,886] Trial 1 finished with value: 0.5476990332697595 and parameters: {'n_estimators': 75, 'max_depth': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.6088191058920795.
[I 2023-11-27 21:30:54,906] Trial 2 finished with value: 0.5818881357039416 and parameters: {'n_estimators': 221, 'max_depth': 4, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.6088191058920795.
[I 2023-11-27 21:31:06,712] Trial 3 finished with value: 0.5619851078584335 and parameters: {'n_estimators': 152, 'max_depth': 3, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.6088191058920795.
[I 2023-11-27 21:31:41,776] Trial 4 finished with value: 0.603299793511089 and parameters: {'n_estimators': 257, 'max_depth': 7, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.608819105

In [61]:
print("Наилучшие значения гиперпараметров RandomForest: {}".format(study_rf_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_rf_smote.best_value))

Наилучшие значения гиперпараметров RandomForest: {'n_estimators': 280, 'max_depth': 9, 'min_samples_leaf': 4}
f1_score на обучающем наборе: 0.613


In [62]:
with mlflow.start_run(nested=True):

    best_params = study_rf_smote.best_params
    best_score = study_rf_smote.best_value
    mlflow.log_params(best_params)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', RandomForestClassifier(**best_params,
                                         random_state=12345))
    ])

    rf_pipe.fit(X_train, y_train)

    pred_proba = rf_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(rf_pipe, 'rf_pipeline_smote')

### 3.2.3 Логистическая регрессия

In [63]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()),
    ('scal', StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [64]:
def optuna_optimize(trial):
    param = {
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga'])
    }

    if param['solver'] == 'lbfgs':
        param['penalty'] = 'l2' 

    if param['solver'] == 'saga':
        param['penalty'] = 'elasticnet' 
        param['C'] = trial.suggest_float('C', 0.01, 100) # какие тут значения?
        param['l1_ratio'] = trial.suggest_float('l1_ratio', 0, 1)
        param['max_iter'] = 1000 

    estimator = LogisticRegression(**param,
                                   random_state=12345)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(log_reg_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [65]:
study_log_reg_smote = optuna.create_study(study_name="LogisticRegression Optuna Optimization SMOTE",
                                    direction='maximize')

[I 2023-11-27 22:34:59,802] A new study created in memory with name: LogisticRegression Optuna Optimization SMOTE


In [66]:
study_log_reg_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 22:35:01,321] Trial 0 finished with value: 0.4931691414132686 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-27 22:35:02,784] Trial 1 finished with value: 0.4931691414132686 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-27 22:35:04,579] Trial 2 finished with value: 0.49305864444463354 and parameters: {'solver': 'saga', 'C': 34.89724643672052, 'l1_ratio': 0.8260559642181673}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-27 22:35:06,387] Trial 3 finished with value: 0.49305864444463354 and parameters: {'solver': 'saga', 'C': 98.58839200684261, 'l1_ratio': 0.47434876890654976}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-27 22:35:07,797] Trial 4 finished with value: 0.4931691414132686 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-27 22:35:09,597] Trial 5 finished with value: 0.49305864444463354 and p

In [67]:
print("Наилучшие значения гиперпараметров Logistic Regression: {}".format(study_log_reg_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_log_reg_smote.best_value))

Наилучшие значения гиперпараметров Logistic Regression: {'solver': 'lbfgs'}
f1_score на обучающем наборе: 0.493


In [68]:
with mlflow.start_run(nested=True):

    best_params = study_log_reg_smote.best_params
    best_score = study_log_reg_smote.best_value
    mlflow.log_params(best_params)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', LogisticRegression(**best_params,
                                     random_state=12345))
    ])

    log_reg_pipe.fit(X_train, y_train)

    pred_proba = log_reg_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(log_reg_pipe, 'log_reg_pipeline_smote')

### 3.2.4 Метод опорных векторов

In [69]:
def optuna_optimize(trial):

    param = {
        'C': trial.suggest_float('C', 0.001, 200)  
    }

    estimator = SVC(**param,
                    random_state=12345,
                    probability=True)

    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(svc_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    
    return score

In [70]:
study_svc_smote = optuna.create_study(study_name="SVC Optuna Optimization SMOTE",
                                direction='maximize')

[I 2023-11-27 22:37:28,914] A new study created in memory with name: SVC Optuna Optimization SMOTE


In [71]:
study_svc_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-27 22:44:45,055] Trial 0 finished with value: 0.5136281780100983 and parameters: {'C': 170.6307160003901}. Best is trial 0 with value: 0.5136281780100983.
[I 2023-11-27 22:49:29,302] Trial 1 finished with value: 0.5286555089764231 and parameters: {'C': 57.41201133605052}. Best is trial 1 with value: 0.5286555089764231.
[I 2023-11-27 22:56:05,385] Trial 2 finished with value: 0.517944026695674 and parameters: {'C': 146.96040661465818}. Best is trial 1 with value: 0.5286555089764231.
[I 2023-11-27 23:03:10,291] Trial 3 finished with value: 0.5147118352176973 and parameters: {'C': 179.32770002739113}. Best is trial 1 with value: 0.5286555089764231.
[I 2023-11-27 23:08:49,800] Trial 4 finished with value: 0.5194447555653008 and parameters: {'C': 105.94337383130765}. Best is trial 1 with value: 0.5286555089764231.
[I 2023-11-27 23:15:03,422] Trial 5 finished with value: 0.5176233872259008 and parameters: {'C': 133.81623792076414}. Best is trial 1 with value: 0.5286555089764231.
[

In [72]:
print("Наилучшие значения гиперпараметров SVC: {}".format(study_svc_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_svc_smote.best_value))

Наилучшие значения гиперпараметров SVC: {'C': 0.6122695267522624}
f1_score на обучающем наборе: 0.576


In [73]:
with mlflow.start_run(nested=True):

    best_params = study_svc_smote.best_params
    best_score = study_svc_smote.best_value
    mlflow.log_params(best_params)

    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', SVC(**best_params,
                      random_state=12345,
                      probability=True))
    ])

    svc_pipe.fit(X_train, y_train)
    pred_proba = svc_pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(svc_pipe, 'svc_pipeline_smote')

In [74]:
colNames = ["XGBoost", "Random Forest", "Logistic Regression", "SVC"]
results = {"Imbalanced":[study_xgb.best_value, '-', '-', '-'],
           "Class Weight":[study_xgb_cw.best_value, 
                           study_rf.best_value, 
                           study_log_reg.best_value, 
                           study_svc.best_value],
           "SMOTE":[study_xgb_smote.best_value, 
                    study_rf_smote.best_value, 
                    study_log_reg_smote.best_value, 
                    study_svc_smote.best_value]}

In [75]:
# результаты метрики f1 score на кроссвалидации:
table = pd.DataFrame.from_dict(results, orient='index', columns=colNames)

In [76]:
table

,XGBoost,Random Forest,Logistic Regression,SVC
Imbalanced,0.591565,-,-,-
Class Weight,0.611431,0.61787,0.491084,0.579064
SMOTE,0.626318,0.612853,0.493169,0.575569


# 4. Тестирование лучшей модели


In [77]:
best_model = mlflow.sklearn.load_model('runs:/0e388b6d46cb442780f28cb35f4e82e1/xgb_pipeline_smote')

In [78]:
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

In [91]:
pred_proba_xgb_smote = best_model.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_xgb_smote))
print('f1 score на тестовой выборке:', f1_score(y_test, np.round(pred_proba_xgb_smote)))
print('Balanced accuracy на тестовой выборке:', balanced_accuracy_score(y_test, np.round(pred_proba_xgb_smote)))

ROC-AUC score на тестовой выборке: 0.864531711989339
f1 score на тестовой выборке: 0.6323529411764707
Balanced accuracy на тестовой выборке: 0.7695584644737188


In [25]:
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('enc',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Geography', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value=-1)),
                                                                  ('pt',
                                                                   PowerTransformer())]),
                                                  ['Balance', 'NumOfProducts',
                                                   'Age', 'EstimatedSalary',
                                                   'Tenure',
                                                   'CreditSco...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=131, n_jobs=None,
                               num_parallel_tree=None, ...))])